### Transformer

![image](../data/image/transformers.jpg)

Transformer分为两个部分
- 编码器：负责将输入序列转换为一种表示
- 解码器：根据这种表示生成输出序列

Transformer分为两个部分使用了大量的自注意力、多头注意力、解码器-解码器注意力  

- 将输入序列的每个元素分别投影到三个不同的向量空间，得到Q，K，V向量。
- 计算Q和K的点积，然后除以一个缩放因子（缩放因子为向量特征维度的平方根），得到注意力分数。
- 用softmax函数对注意力分数进行归一化，得到注意力权重。
- 将注意力权重与对应的V向量相乘，求和，得到自注意力的输出

多头注意力，是让模型能够同时关注输入序列中的多个不同的表示子空间，从而捕捉更丰富的信息  

注意力机制能够大幅提升语言模型性能，原因如下：
- 注意力机制让Transformer能够在不同层次和不同位置捕捉输入序列中的依赖关系。
- 注意力机制使得模型具有强大的表达能力，能够有效处理各种序列到序列任务。
- 可以高度并行化，transformer训练速度得到提升。

由于transformer模型不使用循环神经网络，因此没有位置信息，需要在输入序列添加位置编码，将每个词的位置信息加入到词向量中。  
选用正弦和余弦函数对每个位置进行编码 编码后与词向量进行相加或拼接。因为正余弦函数具有平滑性和保留相对位置信息等优点。


编码器内部由多个相同结构的层堆叠而成，每个层包含两个主要部分：
- 多头注意力
- 前馈神经网络

In [1]:
import torch.nn as nn
import torch
import math
import numpy as np
import torch.nn.functional as F
from copy import deepcopy
from matplotlib import pyplot as plt
import random
from torch.utils.data import DataLoader, Dataset

#### Transformer类组-多头自注意力-缩放点积注意力

ScaledDotProductAttention类  

![image](../data/image/Atten.jpg)

In [2]:
def attention(query, key, value, mask=None):
    '''
    自注意力计算
    query, key, value = [batch, head, token_len, d_k]
    '''
    d_k = query.size(-1)
    # QK^T/srqt{d_k} [batch, head, tokenlen, tokenlen]
    scores = torch.matmul(query, key.transpose(-2,-1)) / math.sqrt(d_k)
    # [batch, head, tokenlen, tokenlen] 
    scores = scores.masked_fill(mask==True, -1e9)
    atten_softmax = torch.softmax(scores, dim=-1)
    return torch.matmul(atten_softmax, value)

#### Transformer类组-多头自注意力-多头注意力

MultiHeadAttention类

![image](../data/image/Attention.jpg)

![image](../data/image/MultiHead.jpg)


In [3]:
class MultiHeadAttention(nn.Module):
    '''
    多头注意力机制
    '''
    def __init__(self, head, d_model):
        '''
        head:多头注意力，头数
        d_model:token Embedding 的维度
        '''
        super(MultiHeadAttention, self).__init__()
        self.d_k = d_model // head
        self.head = head
        self.d_model = d_model
        # 图中的1部分 
        self.linear_query = nn.Linear(d_model, d_model)
        # 图中的2部分 
        self.linear_key = nn.Linear(d_model, d_model)
        # 图中的3部分 
        self.linear_value = nn.Linear(d_model, d_model)
        # 图中的6部分 
        self.linear_out = nn.Linear(d_model, d_model)
        # 残差网络后的归一化
        self.layer_norm = nn.LayerNorm(32)
        
        
    def forward(self, query, key, value, mask=None):
        # 复制一份query做残差用
        clone_query = query.clone()
        # 获得batch大小  
        n_batch = query.size(0)
        # 进行线性变换后，拆分为多头 
        # [batch, token_len, d_model]->[batch,token_len,head, d_k]->[batch, head, token_len, d_k]
        query = self.linear_query(query).view(n_batch, -1, self.head, self.d_k).transpose(1,2)
        key = self.linear_key(key).view(n_batch, -1, self.head, self.d_k).transpose(1,2)
        value = self.linear_value(value).view(n_batch, -1, self.head, self.d_k).transpose(1,2)
        # 计算注意力 图中的4部分
        enc_attention = attention(query, key, value, mask)
        # 注意力计算后，把多头连接到一起 图中的5部分
        # [batch, head, token_len, d_k] -> [batch, token_len, head, d_k] -> [batch, token_len, d_model]
        enc_attention = enc_attention.transpose(1,2).contiguous().view(n_batch, -1, self.head * self.d_k)
        # 连接后，进行一次线性变换，图中的6部分
        outputs = self.linear_out(enc_attention)
        # 最后残差add后，进行归一化
        return self.layer_norm(outputs + clone_query)

#### 前馈网络(Position-wise Feed-Forward Network)
FeedForward

![image](../data/image/FFN.jpg)


Transformer 为什么要加入前馈神经网络？  
- 增强模型表达能力，通过前馈神经网络和自注意力机制的组合，可以学习到不同位置之间的长距离依赖关系。
- 信息融合，将自注意力机制刷出的信息进行融合，每个位置上的信息在经过FFN后，都可以得到一个新的表示。
- 层间传递

In [4]:
class FeedForward(nn.Module):
    def __init__(self, d_model:int, d_ff:int):
        super(FeedForward, self).__init__()
        self.fc_out = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        self.layer_norm = nn.LayerNorm(32)
    
    def forward(self, x):
        x_clone = x.clone()
        out = self.fc_out(x)
        return self.layer_norm(x_clone + out)
        

#### 位置编码

![image](../data/image/pe.jpg)

PE(pos,2i) = sin(pos/10000^{2i/d})  
PE(pos,2i+1) = cos(pos/10000^{2i/d})  

pos:token在句子中的位置，从0到seq_len-1  
d:token embedding的维度  
i:嵌入向量中的每个维度




In [5]:
class PositionEmbedding(nn.Module):
    def __init__(self, voc_size:int, d_model:int, tokenLen:int):
        '''
        dim:d_model token特征的维度
        max_len:句子最大长度
        '''
        super(PositionEmbedding, self).__init__()
        pe = torch.zeros(tokenLen, d_model)
        position = torch.arange(0, tokenLen).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) * torch.tensor(-(math.log(10000.0) / 32))))
        pe[:,0::2] = torch.sin(position * div_term)
        pe[:,1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
        self.embed = nn.Embedding(voc_size, d_model)
        self.embed.weight.data.normal_(0, 0.1)
        
    def forward(self, x):
        emb = self.embed(x)
        emb = emb + self.pe
        return emb

### 编码器

![image](../data/image/encoder.jpg)

transformer 的编码器使用的是自注意力编码，采用的mask机制是填充  
由于数据集中，每句话的长短都不一致，所以训练时，为了可以实现批次训练模型，  
使用<PAD>做为填充，但是模型训练阶段每个词与<PAD>之间没有任何关系，所以在进行  
mask时候需要把填充的部分去掉，也就是不计算注意力。


In [6]:
class EncoderLayer(nn.Module):
    '''
    MultiHeadAttention -> Add & Norm -> Feed Forward -> Add & Norm
    '''
    def __init__(self, head, d_model, d_ff):
        '''
        head:多头注意力的头数
        d_model:词向量的维度
        d_ff:FFN网络中变换的维度
        '''
        super(EncoderLayer, self).__init__()
        self.attn = MultiHeadAttention(head, d_model)
        self.feed_forward = FeedForward(d_model, d_ff)
        
    def forward(self, x, src_mask):
        '''
        编码器阶段采用自注意力编码，所以 query key value 一致
        mask:编码器阶段，模型可以看到全部的输入信息，所以使用填充掩码，去掉因为保证数据长度一致时使用的PAD占位符
        '''
        atten = self.attn(x, x, x, src_mask)
        outputs = self.feed_forward(atten)
        return outputs
        

In [7]:
class Encoder(nn.Module):
    def __init__(self, n_layer, head, d_model, d_ff):
        '''
        n_layer:编码器由多少个编码模块组成
        head:多头注意力的头数
        d_model:词向量的维度
        d_ff:FFN网络中变换的维度
        '''
        super(Encoder, self).__init__()
        self.encoder_layer_list = nn.ModuleList()
        for _ in range(n_layer):
            self.encoder_layer_list.append(EncoderLayer(head, d_model, d_ff))
        
    def forward(self, x, src_mask):
        '''
        x:输入的词
        src_mask:填充掩码
        '''
#         x = self.layer_1(x, src_mask)
#         x = self.layer_2(x, src_mask)
#         x = self.layer_3(x, src_mask)
        for encoder_layer in self.encoder_layer_list:
            x = encoder_layer(x, src_mask)
            
        return x

#### 解码器
![image](../data/image/EncoderDecoder.jpg)

解码器层由2个注意力模块及一个FFN模块组成
第一个注意力模块输入为预测出来的数据    
为了使用模型更有利的训练，往往采用教师强制的方式进行训练，  
在教师强制训练过程中将真实的输出作为下一步的时间的输入。  
为了确保模型在预测当前位置时不会关注到未来信息，需要在第一个自注意力阶段加入后续注意力掩码，这个mask根绝目标序列计算得到 
第二个注意力采用的是encoder和Decoder注意力，query为第一个注意力的输出，key和value为encoder层的输出  
等同于使用解码器去查看在编码器中的权重。mask则采用输入序列计算得到填充掩码

In [8]:
class DecoderLayer(nn.Module):
    '''
    MultiHeadAttention -> Add & Norm -> MultiHeadAttention -> Add & Norm -> Feed Forward -> Add & Norm
    '''
    def __init__(self, head, d_model, d_ff):
        super(DecoderLayer, self).__init__()
        self.decoder_self_atten = MultiHeadAttention(head, d_model)
        self.encoder_decoder_atten = MultiHeadAttention(head, d_model)
        self.feed_forward = FeedForward(d_model, d_ff)
    
    def forward(self, x, y, src_mask, trg_mask):
        '''
        src_mask:根据输入序列计算得到的填充掩码
        trg_mask:根据目标序列计算得到填充掩码+后续掩码
        '''
        # 计算自身的输入序列中的atten
        y = self.decoder_self_atten(y, y, y, trg_mask)
        y = self.encoder_decoder_atten(y, x, x, src_mask)
        ouputs = self.feed_forward(y)
        return ouputs
        

In [9]:
class Decoder(nn.Module):
    def __init__(self,n_layer, head, d_model, d_ff):
        super(Decoder, self).__init__()
        self.decoder_layer_list = nn.ModuleList()
        for _ in range(n_layer):
            self.decoder_layer_list.append(DecoderLayer(head, d_model, d_ff))
            
    def forward(self, x, y, mask_pad_x, mask_tril_y):
        '''
        query:解码器的query
        mask_pad_x:根据输入序列计算得到的填充掩码
        mask_tril_y:根据目标序列计算得到填充掩码+后续掩码
        '''
        for decoder_layer in self.decoder_layer_list:
            y = decoder_layer(x, y, mask_pad_x, mask_tril_y)
        return y
            

#### 创建 mask 包括填充mask & 填充/后续mask  

为了批量训练，保证输入序列长度一致，需要将小于长度的部分使用<PAD>填充，但是在训练模型时，pad部分是无用的，注意力不需要关注的，所以要去掉这一部分  
    
填充mask：去掉使用pad填充的部分
pad的列是true时，意味着任何词对pad的注意力都是0  
但是pad本身对其他词的注意力并不为0，所以pad行不是true  
      
后续mask：对于每个词而言，他只能看到他自己，和他之前的词，而看到之后的词语。

In [10]:
def create_mask_pad(data, pad_idx):
    '''
    填充mask
    data:数据 [batch, seqLen]
    pad_idx: 掩码数
    返回: [batch, 1, 1, seqlen]
    '''
    padding_mask = data == pad_idx
    padding_mask = padding_mask.unsqueeze(1).unsqueeze(2)
    return padding_mask

In [11]:
def create_combined_mask(data, pad_token_id=0):
    #获得token长度
    seqlen = data.size(1)
    padding_mask = create_mask_pad(data, pad_token_id)
    mask = torch.triu(torch.ones(seqlen, seqlen), diagonal=1)
    look_ahead_mask = mask.bool().unsqueeze(0).unsqueeze(0)
    combined_mask = torch.max(padding_mask, look_ahead_mask)
    return combined_mask

In [12]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.embed_x = PositionEmbedding(39, 32, 50)
        self.embed_y = PositionEmbedding(39, 32, 50)
        self.encoder = Encoder(3, 4, 32, 64)
        self.decoder = Decoder(3, 4, 32, 64)
        self.fc_out = nn.Linear(32, voc_size)
        
    def forward(self, x, y):
        mask_pad_x = create_mask_pad(x, wordToidx['<PAD>'])
        mask_combined_mask = create_combined_mask(y, wordToidx['<PAD>'])
        x = self.embed_x(x)
        y = self.embed_y(y)
        x = self.encoder(x, mask_pad_x)
        y = self.decoder(x, y, mask_pad_x, mask_combined_mask)
        y = self.fc_out(y)
        return y
        

#### 生成训练数据

原始数据：从 【0,1,2,3,4,5,6,7,8,9,q,w,e,r,t,y,u,i,o,p,a,s,d,f,g,h,j,k,l,z,x,c,v,b,n,m】随机选取30-48个字符  
目标数据：原始数据的小写字母变成大写，数字变成9-原数字，倒序排列，并且头部重叠一次  
例如：123456qwer->RREWQ345678

In [13]:
words = "<SOS>,<EOS>,<PAD>,0,1,2,3,4,5,6,7,8,9,q,w,e,r,t,y,u,i,o,p,a,s,d,f,g,h,j,k,l,z,x,c,v,b,n,m"

wordToidx = {data:idx for idx, data in enumerate(words.split(','))}
idxToword = {idx:data for idx, data in enumerate(words.split(','))}
lab_wordToidx = {data.upper():idx for idx, data in idxToword.items()}
lab_idxToword = {idx:data.upper() for idx, data in idxToword.items()}
voc_size = len(wordToidx)

In [14]:
def get_data():
    #组成序列的词的列表
    words = [
        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'q', 'w', 'e', 
        'r', 't', 'y', 'u', 'i', 'o', 'p', 'a', 's', 'd', 'f', 'g', 'h',
        'j', 'k', 'l', 'z', 'x', 'c', 'v', 'b', 'n', 'm'
    ]
    # 定义每个单词出现的概率
    p = np.array([
        1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,
        17,18,19,20,21,22,23,24,25,26
    ])
    p = p / p.sum()
    # 随机选取n个
    n = random.randint(30, 48)
    x = np.random.choice(words, size=n, replace=True, p = p)
    x = x.tolist()
    
    def f(i):
        i = i.upper()
        if not i.isdigit():
            return i
        i = 9 - int(i)
        return str(i)
    
    y = [f(_x) for _x in x]
    y = y + [y[-1]]
    y = y[::-1]
    
    x = ["<SOS>"] + x + ["<EOS>"]
    y = ["<SOS>"] + y + ["<EOS>"]
    x = x + ["<PAD>"] * 50
    y = y + ["<PAD>"] * 51
    x = x[:50]
    y = y[:51]
    
    x = [wordToidx[_x] for _x in x]
    y = [lab_wordToidx[_y] for _y in y]
    
    x = torch.LongTensor(x)
    y = torch.LongTensor(y)
    
    return x, y

In [17]:
class _Dataset(Dataset):
    def __init__(self):
        super(_Dataset, self).__init__()
    
    def __len__(self):
        return 10000
    
    def __getitem__(self, index):
        return get_data()

In [18]:
loader = DataLoader(dataset=_Dataset(), batch_size=4, shuffle=True)

In [19]:
model = Transformer()
loss_func = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=2e-3)
# 会在预定的周期（step）按一定的因子（gamma）调整学习率，
# step_size (int): 学习率下调的周期，单位为训练轮次（epochs）。
# gamma (float, optional): 学习率调整的因子，新的学习率等于上一周期学习率乘以 gamma。默认值为 0.1。
sched = torch.optim.lr_scheduler.StepLR(optim, step_size=3, gamma = 0.5)

In [23]:
for epoch in range(1):
    for i, (x, y) in enumerate(loader):
        pred = model(x, y[:,:-1])
        pred = pred.reshape(-1, voc_size)
        y = y[:, 1:].reshape(-1)
        loss = loss_func(pred, y)
        
        # <PAD>的不进行loss操作
        select = y!= lab_wordToidx['<PAD>']
        y = y[select]
        pred = pred[select]
        
        loss = loss_func(pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        if i % 200 == 0:
            #dim=1维度上的最大的数的索引
            pred = pred.argmax(1)
            correct = (pred == y).sum().item()
            accuracy = correct / len(pred)
            print(f"epoch :{epoch},i:{i},loss:{loss.item()}, accuracy:{accuracy}")
            
    sched.step()

epoch :0,i:0,loss:0.00060990359634161, accuracy:1.0
epoch :0,i:200,loss:0.00023534431238658726, accuracy:1.0
epoch :0,i:400,loss:0.0004288146155886352, accuracy:1.0
epoch :0,i:600,loss:0.00025627174181863666, accuracy:1.0
epoch :0,i:800,loss:0.0002982556470669806, accuracy:1.0
epoch :0,i:1000,loss:0.0003540038305800408, accuracy:1.0
epoch :0,i:1200,loss:0.00017349974950775504, accuracy:1.0
epoch :0,i:1400,loss:0.0003045308985747397, accuracy:1.0
epoch :0,i:1600,loss:0.00011168561468366534, accuracy:1.0
epoch :0,i:1800,loss:0.0001509154390078038, accuracy:1.0
epoch :0,i:2000,loss:0.00011977414396824315, accuracy:1.0
epoch :0,i:2200,loss:7.015308801783249e-05, accuracy:1.0
epoch :0,i:2400,loss:6.23313317191787e-05, accuracy:1.0


In [21]:
def predict(x):
    # x = [1, 50]
    model.eval()

    # [1, 1, 50, 50]
    mask_pad_x = create_mask_pad(x, wordToidx['<PAD>'])

    # 初始化输出,这个是固定值
    # [1, 50]
    # [[0,2,2,2...]]
    target = [lab_wordToidx['<SOS>']] + [lab_wordToidx['<PAD>']] * 49
    target = torch.LongTensor(target).unsqueeze(0)

    # x编码,添加位置信息
    # [1, 50] -> [1, 50, 32]
    x = model.embed_x(x)

    # 编码层计算,维度不变
    # [1, 50, 32] -> [1, 50, 32]
    x = model.encoder(x, mask_pad_x)

    # 遍历生成第1个词到第49个词
    for i in range(49):
        # [1, 50]
        y = target

        # [1, 1, 50, 50]
        mask_tril_y = create_combined_mask(y, lab_wordToidx["<PAD>"])

        # y编码,添加位置信息
        # [1, 50] -> [1, 50, 32]
        y = model.embed_y(y)

        # 解码层计算,维度不变
        # [1, 50, 32],[1, 50, 32] -> [1, 50, 32]
        y = model.decoder(x, y, mask_pad_x, mask_tril_y)

        # 全连接输出,39分类
        # [1, 50, 32] -> [1, 50, 39]
        out = model.fc_out(y)

        # 取出当前词的输出
        # [1, 50, 39] -> [1, 39]
        out = out[:, i, :]

        # 取出分类结果
        # [1, 39] -> [1]
        out = out.argmax(dim=1).detach()

        # 以当前词预测下一个词,填到结果中
        target[:, i + 1] = out

    return target

In [24]:
for i, (x, y) in enumerate(loader):
    break

for i in range(4):
    print(i)
    print(''.join([idxToword[i] for i in x[i].tolist()]))
    print(''.join([lab_idxToword[i] for i in y[i].tolist()]))
    print(''.join([lab_idxToword[i] for i in predict(x[i].unsqueeze(0))[0].tolist()]))

0
<SOS>jucoarmlxxzljdmy3nzpvuny7omzmc5xknjbkxkm4i<EOS><PAD><PAD><PAD><PAD><PAD><PAD>
<SOS>II5MKXKBJNKX4CMZMO2YNUVPZN6YMDJLZXXLMRAOCUJ<EOS><PAD><PAD><PAD><PAD><PAD><PAD>
<SOS>II5MKXKBJNKX4CMZMO2YNUVPZN6YMDJLZXXLMRAOCUJ<EOS><EOS><EOS><EOS><EOS><EOS>
1
<SOS>vldtglvbpz3gblv4zkcnlk8axclahh6b6lcf<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
<SOS>FFCL3B3HHALCXA1KLNCKZ5VLBG6ZPBVLGTDLV<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
<SOS>FFCL3B3HHALCXA1KLNCKZ5VLBG6ZPBVLGTDLV<EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS>
2
<SOS>x3yxhg5njsntlxc61lkmehoxyz5vosghbfb7mxvnazmz5<EOS><PAD><PAD><PAD>
<SOS>44ZMZANVXM2BFBHGSOV4ZYXOHEMKL83CXLTNSJN4GHXY6X<EOS><PAD><PAD><PAD>
<SOS>44ZMZANVXM2BFBHGSOV4ZYXOHEMKL83CXLTNSJN4GHXY6X<EOS><EOS><EOS>
3
<SOS>xgvwlumanfrmgnwnsyfmskvjn4lxod7mhx<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
<SOS>XXHM2DOXL5NJVKSMFYSNWNGMRFNAMULWVGX<EOS><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PA